In [1]:
import os
import time
import wandb
import pickle
import torch
import faiss
import numpy as np
import pandas as pd
from typing import List, Dict, Any
import sys
sys.path.append('/content/drive/Othercomputers/kyei_mac_m3_pro/MediMaven')
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset, DatasetDict
from src.utils import get_mongo_connection
from pipelines.llama3_finetune_pipeline import (
    fetch_finetune_data,chunk_llm_docs, 
    prepare_instruction_text, tokenize_data, split_dataset)


In [3]:
torch.cuda.empty_cache()

In [ ]:
# def save_dataset_splits_as_wandb_artifact(dataset_splits: dict, artifact_name: str, project: str, run_name: str = "log_dataset_splits"):

#     # Start a W&B run
#     run = wandb.init(project=project, name=run_name)

#     # Create artifact
#     artifact = wandb.Artifact(name=artifact_name, type="split_dataset")

#     # Save each split to disk and add to artifact
#     os.makedirs("../tmp_dataset", exist_ok=True)
#     for split_name, dataset in dataset_splits.items():
#         split_path = f"tmp_dataset/{split_name}"
#         dataset.save_to_disk(split_path)
#         artifact.add_dir(local_path=split_path, name=split_name)

#     # Log artifact
#     run.log_artifact(artifact)
#     run.finish()

#     # Cleanup
#     import shutil
#     shutil.rmtree("tmp_dataset")

In [1]:
# save_dataset_splits_as_wandb_artifact(split_ds, "Split_ds", "MediMaven-Data")

In [2]:
def load_dataset_splits_from_wandb_artifact(project:str) -> DatasetDict:
    """
    Loads dataset splits from a W&B artifact into a HuggingFace DatasetDict.
    """
    run = wandb.init(project=project, job_type="load_dataset")

    artifact = run.use_artifact('phade160/MediMaven-LLM-Finetuning/Split_ds:v0', type='split_dataset')
    artifact_dir = artifact.download()

    splits = {}
    for split in ["train", "validation", "test"]:
        path = os.path.join(artifact_dir, split)
        if os.path.exists(path):
            splits[split] = Dataset.load_from_disk(path)

    run.finish()
    return DatasetDict(splits)

In [3]:
loaded_splits = load_dataset_splits_from_wandb_artifact(project="MediMaven-LLM-Finetuning")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: phade160 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact Split_ds:v0, 193.76MB. 9 files... 
wandb:   9 of 9 files downloaded.  
Done. 0:0:3.0


In [4]:
loaded_splits['train'].select(range(1000))

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

In [2]:
querry = "what causes hypertension?"

## 1. Further Quantization with GPTQModel

In [10]:
!pip install -v gptqmodel

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [5]:
from gptqmodel import GPTQModel, QuantizeConfig



INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


In [6]:

from gptqmodel import GPTQModel, QuantizeConfig

model_dir = "../models/llama_finetuned/merged_model_fp16"
save_dir = "../models/llama_finetuned/gptqmodel_4bit"
calibration_dataset = loaded_splits['train'].select(range(1000))

quant_config = QuantizeConfig(bits=4, group_size=128)

# This loads the original full-precision model and prepares it for quantization
model = GPTQModel.load(model_dir, quant_config)

# Quantize using the calibration dataset
model.quantize(calibration_dataset, batch_size=8)

# Save the quantized model
model.save(save_dir)


INFO  Estimated Quantization BPW (bits per weight): 4.2875 bpw, based on [bits: 4, group_size: 128]
INFO  Loader: Auto dtype (native float16): `torch.float16`                     


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').


Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').
INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

INFO  Kernel: loaded -> `[]`                                                   
INFO  Packing Kernel: Auto-selection: adding candidate `TritonV2QuantLinear`   
INFO  {'process': 'gptq', 'layer': 0, 'module': 'self_attn.k_proj', 'loss': '0.20497', 'damp': '0.01000', 'time': '2.207', 'fwd_time': '8.249'}
INFO  {'process': 'gptq', 'layer': 0, 'module': 'self_attn.v_proj', 'loss': '0.00201', 'damp': '0.01000', 'time': '1.465', 'fwd_time': '8.249'}
INFO  {'process': 'gptq', 'layer': 0, 'module': 'self_attn.q_proj', 'loss': '0.30193', 'damp': '0.01000', 'time': '1.488', 'fwd_time': '8.249'}
INFO  {'process': 'gptq', 'layer': 0, 'module': 'self_attn.o_proj', 'loss': '0.00019', 'damp': '0.01000', 'time': '1.464',

In [8]:
querry = "what causes hypertension?"

## 1. Retriever

In [ ]:
## 1. Retriever
embeder = SentenceTransformer(embed_mode_name)
querry_embedding = embeder.encode(querry)
querry_embedding

## GPU INFERENCE TEST

In [7]:
# test post-quant inference
model = GPTQModel.load(save_dir)
result = model.generate("Uncovering deep insights begins with")[0] # tokens
print(model.tokenizer.decode(result)) # string output

from_quantized: adapter: None
INFO  Loader: Auto dtype: `torch.float16` due to inference mode. If you wish to use `bfloat16`, please pass in `torch_dtype` arg to `loader()`.
INFO  Estimated Quantization BPW (bits per weight): 4.2875 bpw, based on [bits: 4, group_size: 128]
INFO  Optimize: Auto enabling flash attention2                                 
INFO   Kernel: Auto-selection: adding candidate `MarlinQuantLinear`            
INFO   Kernel: Auto-selection: adding candidate `ExllamaV2QuantLinear`         
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`           
INFO   Kernel: Auto-selection: adding candidate `TritonV2QuantLinear`          
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             
INFO  Kernel: candidates -> `[MarlinQuantLinear, ExllamaV2QuantLinear, ExllamaQuantLinear, TritonV2QuantLinear, TorchQuantLinear]`
INFO  Kernel: selected -> `MarlinQuantLinear`.                                 
INFO  Format: Skipped v1 to v2 conv

INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').


Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').
INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

INFO  Model: Auto-fixed `generation_config` mismatch between model and `generation_config.json`.
INFO  Model: Updated `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

INFO  Kernel: loaded -> `[MarlinQuantLinear]`                                  
<|begin_of_text|>Uncovering deep insights begins with a deep understanding of your audience. We use data and analytics to understand your customers, their behaviors,


In [12]:
from sentence_transformers import SentenceTransformer
from src.utils import cosine_similarity,l2_distance
import xgboost as xgb


In [10]:
#Metedata
import json
with open("../models/faiss_index_data.json", "rb") as f:
    docs_metadata = json.load(f) 
docs_metadata

[{'context_id': '0',
  'faiss_id': 0,
  'context': 'alymphoid cystic thymic dysgenesis,congenital alopecia and nail dystrophy associated with severe functional T cell immunodeficiency,Pignata Guarino syndrome,winged helix deficiency T cell immunodeficiency, congenital alopecia, and nail dystrophy What is are T cell immunodeficiency, congenital alopecia, and nail dystrophy ?'},
 {'context_id': '1',
  'faiss_id': 1,
  'context': 'alymphoid cystic thymic dysgenesis,congenital alopecia and nail dystrophy associated with severe functional T cell immunodeficiency,Pignata Guarino syndrome,winged helix deficiency T cell immunodeficiency, congenital alopecia, and nail dystrophy How many people are affected by T cell immunodeficiency, congenital alopecia, and nail dystrophy ?'},
 {'context_id': '2',
  'faiss_id': 2,
  'context': 'alymphoid cystic thymic dysgenesis,congenital alopecia and nail dystrophy associated with severe functional T cell immunodeficiency,Pignata Guarino syndrome,winged heli

In [22]:
# 1. embed
def embed(query, embed_mode_name ="sentence-transformers/all-MiniLM-L6-v2"):
    embeder = SentenceTransformer(embed_mode_name)
    query_embedding = embeder.encode(query)
    return query_embedding
    

#2. Retrieve
def retrieve(query_emb, model_path, metadata = docs_metadata):
    retriever = faiss.read_index(model_path)
    dist, idx = retriever.search(np.expand_dims(query_emb, 0), 5)
    docs = []
    for d, i in zip(dist[0], idx[0]):
        doc = { 'doc_id': i,
                "context_id": metadata[i]["context_id"],
                "score": d,
                "context": metadata[i]["context"]}
        docs.append(doc)
    return docs

#3 rerank

def rerank(ranker_path, docs, querry_embedding ):
    ranker = xgb.XGBRanker()
    ranker.load_model(ranker_path)
    #build features
    docs_embeddings = [embed(query=doc['context']) for doc in docs]
    cosims = [cosine_similarity(querry_embedding, doc_emb) for doc_emb in docs_embeddings]
    l2s = [l2_distance(querry_embedding, doc_emb) for doc_emb in docs_embeddings]
    lens = [len(doc['context'].split()) for doc in docs]
    features = np.array([cosims, l2s, lens]).T
    #make preds of ranks
    scores = ranker.predict(features)
    for doc, score in zip(docs, scores):
        doc["ltr_score"] = float(score)

    reranked_docs= sorted(docs, key=lambda x: x['ltr_score'], reverse=True)
    return reranked_docs

def build_prompt(querry: str, context: str) -> str:
    sys_msg = (
        "You are a helpful medical AI assistant. "
        "Provide clear and concise answers based on the given question and context."
        "the answer is after the [/INST] tag so generate and summerize evrything after the [/INST] tag"
    )

    # Example format for Llama instructions:
    prompt = (
        f"[INST] <<SYS>>\n{sys_msg}\n<<SYS>>\n"
        f"Question: {querry}\n"
        f"Context: {context}\n"
        "[/INST]"
    )
    return prompt
#4. generate
def generate(model_path, query, context, device='cuda' ):
    model = GPTQModel.load(model_path)
    prompt = build_prompt(query, context)
    
    inputs = model.tokenizer(prompt, return_tensors="pt").to(device)
    output = model.generate(**inputs,
                            max_new_tokens=256,
                            temperature=0.7,
                            do_sample=True)[0]
    response = model.tokenizer.decode(output, skip_special_tokens=True)
    return response.split("[/INST]")[-1].strip()



In [16]:
querry_embedding = embed(query=querry)
querry_embedding

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Use pytorch device_name: cuda


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([-1.07108271e-02,  6.39325082e-02, -2.17748731e-02,  6.37408867e-02,
        5.10996208e-02,  7.54497014e-03,  2.40730867e-02,  2.48579122e-02,
       -3.72231901e-02, -2.40275692e-02, -5.84573252e-03, -5.30484729e-02,
       -1.95338968e-02, -2.40437686e-02, -4.03857529e-02, -9.75393318e-03,
       -7.33973011e-02, -3.04172765e-02,  3.00706495e-02,  2.97163650e-02,
        3.17584872e-02,  5.93150072e-02, -4.10248078e-02,  3.11656073e-02,
       -1.98439378e-02,  3.16797309e-02,  3.24705318e-02, -8.07417408e-02,
       -9.29016806e-03, -1.42471259e-02,  2.60793082e-02, -2.27623824e-02,
       -4.33902964e-02,  6.57521486e-02,  1.17193628e-02,  7.83134922e-02,
        1.36830630e-02, -1.94479115e-02, -7.36206472e-02,  5.69709986e-02,
       -1.54990330e-03, -9.11833569e-02, -7.45328888e-02,  5.94863333e-02,
        3.20203118e-02,  5.00806309e-02, -4.46898490e-02,  5.12904599e-02,
        3.00790630e-02,  5.31335473e-02, -9.88918021e-02,  5.96194807e-03,
       -2.72183493e-03, -

In [19]:
#retrieve
faiss_model_path =  "../models/faiss_index.bin"
docs = retrieve(query_emb=querry_embedding,model_path=faiss_model_path, metadata=docs_metadata)
docs

[{'doc_id': np.int64(11935),
  'context_id': '11211',
  'score': np.float32(0.6217982),
  'context': 'Hypertension, commonly known as high blood pressure, is a condition where the force of the blood against the artery walls is consistently too high. Hello doctor,\nMy friend is 40 years old. He was diagnosed with hypertension recently. What are the risk factors of this hypertension? What are the adverse health effects faced due to hypertension? What is the importance of regular blood pressure monitoring? What complications occur due to uncontrolled hypertension and how to manage it?\nPlease help.\nThank you.'},
 {'doc_id': np.int64(4608),
  'context_id': '4579',
  'score': np.float32(0.62611127),
  'context': 'Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,\nI am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing fa

In [23]:
# rerank
ranker_path = "../models/ltr_best_model.json" 


rerank_docs = rerank(ranker_path=ranker_path, docs=docs, querry_embedding=querry_embedding)
rerank_docs

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda


Use pytorch device_name: cuda


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Use pytorch device_name: cuda


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[{'doc_id': np.int64(4608),
  'context_id': '4579',
  'score': np.float32(0.62611127),
  'context': 'Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,\nI am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing factors of hypertension. I am interested in receiving guidance on lifestyle modifications that can help manage and control hypertension. I would like to know about how to monitor my blood pressure at home and a schedule for follow up appointments to track my progress. I would also like to know some preventive measures that might help maintain my overall cardiovascular health. Please help.\nThank you.',
  'ltr_score': 4.9730305671691895},
 {'doc_id': np.int64(16588),
  'context_id': '15426',
  'score': np.float32(0.675048),
  'context': 'Hypertension, commonly known as high blood pressure, is a medical condition c

In [27]:
context = "\n".join(d["context"] for d in rerank_docs[:3])
print(context)

Hypertension can be influenced by factors such as genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Hi doctor,
I am a 40 year old female. Recently, I was diagnosed with hypertension. I would like to know the potential causes and contributing factors of hypertension. I am interested in receiving guidance on lifestyle modifications that can help manage and control hypertension. I would like to know about how to monitor my blood pressure at home and a schedule for follow up appointments to track my progress. I would also like to know some preventive measures that might help maintain my overall cardiovascular health. Please help.
Thank you.
Hypertension, commonly known as high blood pressure, is a medical condition characterized by elevated pressure in the arteries. Hello doctor,
I am a 34 year old male diagnosed with hypertension. Kindly guide.
Thank you in advance.
High blood pressure, or hypertension, is a medical condition characterized by elevated p

In [28]:
answer = generate(model_path=save_dir, query=querry, context=context)

from_quantized: adapter: None
INFO  Loader: Auto dtype: `torch.float16` due to inference mode. If you wish to use `bfloat16`, please pass in `torch_dtype` arg to `loader()`.
INFO  Estimated Quantization BPW (bits per weight): 4.2875 bpw, based on [bits: 4, group_size: 128]
INFO  Optimize: Auto enabling flash attention2                                 
INFO   Kernel: Auto-selection: adding candidate `MarlinQuantLinear`            
INFO   Kernel: Auto-selection: adding candidate `ExllamaV2QuantLinear`         
INFO   Kernel: Auto-selection: adding candidate `ExllamaQuantLinear`           
INFO   Kernel: Auto-selection: adding candidate `TritonV2QuantLinear`          
INFO   Kernel: Auto-selection: adding candidate `TorchQuantLinear`             
INFO  Kernel: candidates -> `[MarlinQuantLinear, ExllamaV2QuantLinear, ExllamaQuantLinear, TritonV2QuantLinear, TorchQuantLinear]`
INFO  Kernel: selected -> `MarlinQuantLinear`.                                 
INFO  Format: Skipped v1 to v2 conv

INFO:tokenicer.tokenicer:Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').


Tokenicer: Auto fixed pad_token_id=128004 (token='<|finetune_right_pad_id|>').
INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ]
}

INFO  Model: Auto-fixed `generation_config` mismatch between model and `generation_config.json`.
INFO  Model: Updated `generation_config`: GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "temperature": 0.6,
  "top_p": 0.9
}

INFO  Kernel: loaded -> `[MarlinQuantLinear]`                                  


In [30]:
print(answer)

Hello,
Welcome to icliniq.com.
I read your query and can understand your concern.
Hypertension is a common health concern that can be managed with lifestyle modifications and, if necessary, medication. Factors that contribute to high blood pressure include genetics, age, obesity, lack of physical activity, excessive salt intake, and stress. Lifestyle modifications, such as maintaining a healthy diet, engaging in regular exercise, managing stress, and monitoring blood pressure regularly, can help manage hypertension. Monitoring blood pressure at home can be done using a blood pressure monitor. It is recommended to track blood pressure at the same time each day, ideally in the morning and evening, for at least three to five days before a doctor s appointment. Regular follow up appointments with a healthcare provider are necessary to monitor blood pressure and adjust treatment plans as needed. Preventive measures for maintaining cardiovascular health include maintaining a healthy weight, 